In [19]:
from parallel_api_call import *

raw_path = "./libero/libero/"
output_path = "./output/"
raw_jsons = ["libero_10.json", "libero_goal.json", "libero_object.json", "libero_spatial.json"]

In [17]:
SYS_PROMPT="You are a helpful assistant."


ONE_PROMPT = """
Please generate 3 different revised versions of the sentence below. Each time select **one** verb, noun, or adjective from the sentence below at random and replace it with a synonym or a similar word, ensuring that the overall meaning of the sentence remains unchanged.
SENTENCE: {sentence}
OUTPUT FORMAT (JSON):
{{
    "RAW": (a string), copy of the input sentence,
    "REVISED": [
        (a list of strings), 3 different revised versions
    ]
}}
"""

TWO_PROMPT = """
Please generate 3 different revised versions of the sentence below. Each time choose either **two random words** (a verb, noun, or adjective) or **one phrase** in the sentence below and replace them with similar expressions, while keeping the sentence’s overall meaning intact.
SENTENCE: {sentence}
OUTPUT FORMAT (JSON):
{{
    "RAW": (a string), copy of the input sentence,
    "REVISED": [
        (a list of strings), 3 different revised versions
    ]
}}
"""

ALL_PROMPT = """
Please generate 3 different revised versions of the sentence below. Each time rewrite the entire sentence with significant changes in structure and wording, while ensuring that the overall meaning remains the same. you can change like: please xxx? or OpenVLA, can you xxx?
SENTENCE: {sentence}
OUTPUT FORMAT (JSON):
{{
    "RAW": (a string), copy of the input sentence,
    "REVISED": [
        (a list of strings), 3 different revised versions
    ]
}}
"""

PROMPT_MAP={
    "one":ONE_PROMPT,
    "two":TWO_PROMPT,
    "all": ALL_PROMPT,
}

In [14]:
raw_data = {}
for json_name in raw_jsons:
    with open(f"{raw_path}{json_name}", "r") as f:
        obj = json.load(f)
        inss = obj.keys()
        raw_data[json_name] = list(inss)

In [20]:
for mode in PROMPT_MAP.keys():
    prompt = PROMPT_MAP[mode]
    for json_name in raw_jsons:
        inputs = [prompt.format(sentence = ins) for ins in raw_data[json_name]]
        responses = await async_process_queries(SYS_PROMPT, inputs, temperature=0.5, top_p=0.9)
        converted = [str_to_json(response) for response in responses]
        # 检查转换
        failidxs = []
        for idx, obj in enumerate(converted):
            if obj is None:
                failidxs.append(idx)
        # 重新请求
        if len(failidxs) >0:
            print("request again")
            converted = [None for _ in range(len(failidxs))]
            while None in converted:
                responses = await async_process_queries(SYS_PROMPT, inputs, temperature=0.5, top_p=0.9)
            converted = [str_to_json(response) for response in responses]
        # 保存
        tuples = [(raw_data[json_name][i], converted[i]["REVISED"]) for i in range(len(inputs))]
        save_dict = {k:v for k,v in tuples}
        with open(f"{output_path}{mode}_{json_name}", "w") as f:
            json.dump(save_dict, f, indent=4)
